In [6]:
consumer_key = 'NVNiYbndhDGhWop1L8XvOrrbH'
consumer_secret = 'iu4x0cPVfRUMbFHAJop0kMTi338UyGFATcRk3gBVpEo5BpvCck'
access_token = '104533817-0BN5I3TLkI9Nky6JgRpnb2H1IMXwDkUJuS8p6Nyt'
access_token_secret = '1IRNO1DbmKgmJ8xqiOl3eK7QGwcRI5BNuhNcH04QmQWt1'

In [7]:
from tweepy import API
from tweepy.auth import OAuthHandler
from tweepy import Cursor
import csv
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = API(auth)

# Open/Create a file to append data
csvFile = open('result-oscars-with-header.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile, lineterminator='\n')
csvWriter.writerow(["date","tweet","fav"])
for tweet in Cursor(api.search, q="#oscars", lang='en', count='100000').items() :
   if(tweet.retweet_count > 10 & tweet.retweeted == False) : 
        try:
           csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.favorite_count])
        except:
            continue

In [8]:
import urllib
import urllib.request
import ast
data = urllib.parse.urlencode({"text": "I'm a very good boy "}) 
binary_data = data.encode('utf-8')
u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", binary_data)
the_page = u.read()
print(the_page)

b'{"probability": {"neg": 0.22531846855219551, "neutral": 0.084284385065714951, "pos": 0.77468153144780449}, "label": "pos"}'


In [23]:
import urllib
import urllib.request
import json
import pandas as pd
import re

dataf = pd.read_csv('result-oscars-with-header.csv')
results = []
for index, row in dataf.iterrows():
    print(row['tweet'])
    data = urllib.parse.urlencode({"text": re.sub(r"http\S+", "", row['tweet'])}) 
    binary_data = data.encode('utf-8')
    u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", binary_data)
    result = json.loads(u.read().decode('utf-8'))
    results.append(result['label'])
    print(result['label'])
dataf['label-ext'] = results
dataf.to_csv('result-oscars-with-labelled.csv')

b'RT @WhiteHouse: \xe2\x80\x9cClimate change is real...let\'s not take this planet for granted."\n\nThank you, @LeoDiCaprio. #ActOnClimate #Oscars https://\xe2\x80\xa6'


HTTPError: HTTP Error 503: Service Unavailable

In [14]:
testdf = pd.read_csv('result-oscars-with-header.csv')
testdf.columns
for idx, row in testdf.iterrows():
    testdf.loc[idx,'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",row['tweet']).split())
testdf.to_csv('result-oscars-test.csv')
testdf.columns

Index(['date', 'tweet', 'fav', 'Unnamed: 3'], dtype='object')

In [59]:
testdf = pd.read_csv('result-oscars-train.csv')
testdf = testdf.drop_duplicates(subset='tweet')
testdf.to_csv('result-oscars-train.csv')

In [60]:
normalizedDf = testdf.groupby('label').head(50)
print(normalizedDf.groupby('label').count())
print(normalizedDf.size)
normalizedDf.to_csv('result-oscars-normalized-train.csv')

       Unnamed: 0  Unnamed: 0.1  date  tweet  fav
label                                            
funny          50            50    50     50   50
info           50            50    50     50   50
neg            50            50    50     50   50
pos            50            50    50     50   50
1200


In [61]:
testdf = pd.read_csv('result-oscars-normalized-train.csv')
testdf = testdf.drop_duplicates(subset='tweet')
testdf = testdf[['tweet','label']]
tweets = []
for index, row in testdf.iterrows():
    words = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",row['tweet']).split())
    #words = re.sub(r"http\S+", "", row['tweet'])
    #words = row['tweet']
    sentiment = row['label']
    words_filtered = [e.lower() for e in words.split() if len(e) >= 4]
    tweets.append((words_filtered, sentiment))
    
tweets


[(['catch'], 'funny'),
 (['teacher',
   'where',
   'your',
   'homework',
   'where',
   'leonardo',
   'dicaprio',
   'oscar'],
  'funny'),
 (['presenter',
   'oscar',
   'goes',
   'nleo',
   'starts',
   'slowly',
   'stage',
   'npresenter',
   'damn',
   'daniel',
   'back'],
  'funny'),
 (['performance'], 'info'),
 (['michael', 'jordan', 'looking', 'sharp', 'tonight'], 'funny'),
 (['nutshell'], 'info'),
 (['wouldn',
   'funny',
   'fifty',
   'years',
   'someone',
   'made',
   'movie',
   'about',
   'leonardo',
   'dicaprio',
   'actor',
   'that',
   'played'],
  'funny'),
 (['audrey', 'hepburn', 'grace', 'kelly', 'backstage'], 'info'),
 (['fact',
   'from',
   'dicaprio',
   'animal',
   'agriculture',
   'source',
   'greenhouse',
   'gasses'],
  'info'),
 (['best', 'thing', 'stop'], 'info'),
 (['climate',
   'change',
   'most',
   'urgent',
   'threat',
   'facing',
   'entire',
   'species',
   'agree',
   'with'],
  'info'),
 (['only', 'bradley', 'longer', 'best', 'pho

In [77]:
import nltk
from nltk.text import TextCollection
from collections import OrderedDict
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words
def get_tfidf_word_features(tweets):
    tweets_text = [word for (word,sentiment) in tweets]
    tweets_coll = TextCollection(tweets_text)
    wordlist = get_words_in_tweets(tweets)
    wordlist_freq = nltk.FreqDist(wordlist)
    word_features = {}
    for word in set(wordlist):
        if word:
            wordlist_freq[word] =  (tweets_coll.idf(word)*wordlist_freq[word]) if(tweets_coll.idf(word)*wordlist_freq[word]) else 0
    return wordlist_freq.keys()
    
#The list of word features need to be extracted from the tweets. It is a list with every distinct words ordered 
#by frequency of appearance. We use the following function to get the list plus the two helper functions.
word_features = get_word_features(get_words_in_tweets(tweets))
#word_features = get_tfidf_word_features(tweets)

#To create a classifier, we need to decide what features are relevant.
#To do that, we first need a feature extractor. The one we are going to use returns a dictionary
#indicating what words are contained in the input passed. Here, the input is the tweet. 
#We use the word features list defined above along with the input to create the dictionary.
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

training_set = nltk.classify.apply_features(extract_features, tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)


In [78]:
print (classifier.show_most_informative_features(100))
#print(training_set)

Most Informative Features
         contains(oscar) = True            funny : info   =      6.3 : 1.0
          contains(gaga) = True              pos : info   =      4.3 : 1.0
          contains(lady) = True              pos : info   =      4.3 : 1.0
         contains(facts) = True              pos : info   =      3.7 : 1.0
       contains(tonight) = True            funny : pos    =      3.7 : 1.0
         contains(right) = True            funny : pos    =      3.7 : 1.0
          contains(this) = True            funny : neg    =      3.4 : 1.0
          contains(2016) = True             info : pos    =      3.0 : 1.0
        contains(oscars) = True              neg : funny  =      3.0 : 1.0
          contains(kate) = True              pos : info   =      3.0 : 1.0
          contains(more) = True            funny : pos    =      3.0 : 1.0
          contains(need) = True              neg : pos    =      3.0 : 1.0
          contains(have) = True              neg : info   =      3.0 : 1.0

In [91]:
dataf = pd.read_csv('result-oscars-test.csv')
df = pd.read_csv('result-oscars-test.csv')
results2 = []
for index, row in dataf.iterrows():
    #results2.append(classifier.classify(extract_features(re.sub(r"http\S+", "", row['tweet']))))
    tweet = (re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",row['tweet'].lower()).split())
    print (row['tweet'])
    print(tweet)
    result = classifier.classify(extract_features(tweet))

    print(result)
    results2.append(result)
df['classifier'] = results2
df.to_csv('result-oscars-with-labeled.csv')

b'RT @Justring4: Someone please tell us what Sylvester Stallone is doing? #Oscars https://t.co/RVdUbCyXgT'
['b', 'rt', 'someone', 'please', 'tell', 'us', 'what', 'sylvester', 'stallone', 'is', 'doing']
funny
b'RT @BradYpnexthome: The Big Short? Not in Canada - YP NextHome https://t.co/PeQ050P8Px #oscars #thebigshort #mortgage #financialplanning ht\xe2\x80\xa6'
['b', 'rt', 'the', 'big', 'short', 'not', 'in', 'canada', 'yp', 'nexthome', 'ht', 'xe2', 'x80', 'xa6']
info
b"Sup with Leo's chin? #Oscars https://t.co/qlED3p0wpI"
['b', 'sup', 'with', 'leo', 's', 'chin']
pos
b'Someone please tell us what Sylvester Stallone is doing? #Oscars https://t.co/RVdUbCyXgT'
['b', 'someone', 'please', 'tell', 'us', 'what', 'sylvester', 'stallone', 'is', 'doing']
funny
b'RT @ChanelVsWalmart: Chanel Vs. Walmart #Oscars https://t.co/uMVKXX4vZz'
['b', 'rt', 'chanel', 'vs', 'walmart']
funny
b'RT @SweetLoveMsg: JACK &amp; ROSE FOREVER. \xe2\x9d\xa4\xef\xb8\x8f\xe2\x9a\x93\xef\xb8\x8f\xf0\x9f\x92\x99 #Oscars htt

In [87]:
result = classifier.classify(extract_features(['racist','white', 'never']))
result 

'neg'

In [96]:
dataf = pd.read_csv('result-oscars-with-header.csv')
df = pd.read_csv('result-oscars-with-header.csv')
results2 = []
for index, row in dataf.iterrows():
    #results2.append(classifier.classify(extract_features(re.sub(r"http\S+", "", row['tweet']))))
    tweet = (re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",row['tweet'].lower()).split())
    result = classifier.classify(extract_features(tweet))
    print (row['tweet'])
    print(result)
    results2.append(result)
df['classifier'] = results2
df.to_csv('result-oscars-with-labeled.csv')

b'RT @WhiteHouse: \xe2\x80\x9cClimate change is real...let\'s not take this planet for granted."\n\nThank you, @LeoDiCaprio. #ActOnClimate #Oscars https://\xe2\x80\xa6'
info
b"RT @Courtney: can't tell you how much I adore this queen @ladygaga \xe2\x9d\xa4\xf0\x9f\x91\x91 \xe2\x9d\xa4  #tbt #oscars #vanityfair #givenchy #neillane https://t.co/nt9PtZ9g\xe2\x80\xa6"
funny
b'RT @MOVIEMEMORlES: JACK &amp; ROSE FOREVER. \xe2\x9d\xa4\xef\xb8\x8f\xe2\x9a\x93\xef\xb8\x8f\xf0\x9f\x92\x99 #Oscars https://t.co/ZFOnAjUWWp'
info
b'#Oscars winner "SPOTLIGHT" closes tomorrow night @TheLittleRoch #ROC https://t.co/o4egZ3HMti'
pos
b'RT @ChanelVsWalmart: Chanel Vs. Walmart #Oscars https://t.co/uMVKXX4vZz'
funny
b"RT @TheEllenShow: If only Bradley's arm was longer. Best photo ever. #oscars http://t.co/C9U5NOtGap"
funny
b'RT @JLo: #TwitterExclusive behind the scenes with the dream team... #Oscars http://t.co/Tf2yDuNoup'
pos
b"RT @Trocive_: 'WHEN IS IT MY TURN MOM?' - LEO #Oscars https://t.co/s4g8uJh1sk"
in

KeyboardInterrupt: 